SAGEConv()实现了 GraphSAGE 算法，用于在大型图数据上进行节点分类等任务

# 采样

需采样数量>邻居数量：对某些邻居进行多次采样  
需采样数量<邻居数量：随机采取某些邻居  

就是因为采样，而不是所有都考虑，所以在大图上，可以实现  


1.将所有指向i的节点 聚合得到h'  

聚合类型：  
(1)'mean':取自身和邻居节点的特征平均    
(2) 'gcn': 归一化临界矩阵
h'=mean(hu/sqpt(du*dv))

(3)'pool':通过MLP对每一个hu做变换，然后取最大值

(4) 'lstm': 把邻居节点放到lstm网络中(目前不大懂) 

2.h'和h进行连接并执行激活函数     
    ReLU(W*concat(h,h'))

3.规范化  
通常为None

In [48]:
import dgl
import numpy as np
import torch as th
from dgl.nn import SAGEConv

In [49]:
# Case 1: Homogeneous graph
g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
g = dgl.add_self_loop(g)
feat = th.ones(6, 10)
conv = SAGEConv(10, 2, 'pool')
res = conv(g, feat)

print(res)

tensor([[ 0.7602, -0.9714],
        [ 0.7602, -0.9714],
        [ 0.7602, -0.9714],
        [ 0.7602, -0.9714],
        [ 0.7602, -0.9714],
        [ 0.7602, -0.9714]], grad_fn=<AddBackward0>)


In [50]:
u = [0, 1, 0, 0, 1]
v = [0, 1, 2, 3, 2]

g = dgl.heterograph({('_U', '_E', '_V'):(u, v)})

u_fea = th.rand(2, 5)
v_fea = th.rand(4, 10)

# 定义GraphSAGE卷积层
conv = SAGEConv((5, 10), 2, 'mean')

# 对图进行卷积操作
res = conv(g, (u_fea, v_fea))

print(res)

tensor([[ 1.0660, -0.5623],
        [ 1.9903,  0.6966],
        [ 0.9700, -0.0907],
        [ 1.2641, -0.1373]], grad_fn=<AddBackward0>)
